In [2]:
import os
import glob
import trimesh
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

2023-12-10 12:32:16.883357: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


##### LOAD THE DATASET

In [3]:
tf.random.set_seed(1234)
DATA_DIR = tf.keras.utils.get_file(
    "modelnet.zip",
    "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip",
    extract=True,
)
DATA_DIR = os.path.join(os.path.dirname(DATA_DIR), "ModelNet10")

##### CHOOSE THE CLASSES
2 classes that we use to train PointNet model, there is no much difference in which two classes do you choose 

In [4]:
#chair_t = 'chair_bathtub'
chair_t = 'dresser_chair' 
# chair_t = 'sofa_table'
#chair_t = 'chair_6'
d_max = 1024 #  size of latent space 
batch = 1
BATCH_SIZE  = batch # batch size, the best results for batch = 1
num_points = 512  # num_points = sample_size 
NUM_CLASSES = 2
NUM_POINTS = num_points
epochs = 20 # number of epochs

In [5]:
current_directory = os.getcwd()

weights_path = os.path.join(current_directory, 'new_weights/') # where to save weights

In [6]:
def parse_dataset(num_points=num_points):

    train_points = []
    train_labels = []
    test_points = []
    test_labels = []
    
    
    class_map = {}
    folders_1 = glob.glob(os.path.join(DATA_DIR, "[!README]*"))
    folders = [None]*2
    if chair_t == 'chair_bathtub':
        folders[0] = folders_1[2] #chair
        folders[1] = folders_1[4] #bathtub
    elif chair_t == 'dresser_chair':
        folders[0] = folders_1[0] #dresser
        folders[1] = folders_1[2] #chair
    elif chair_t == 'sofa_table':
        folders[0] = folders_1[3] #sofa
        folders[1] = folders_1[6] #table
    elif chair_t == 'chair_6':
        folders[0] = folders_1[0] 
        folders[1] = folders_1[1] 
        folders[2] = folders_1[2] 
        folders[3] = folders_1[3] 
        folders[4] = folders_1[4] 
        folders[5] = folders_1[5] 


    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        # store folder name with ID so we can retrieve later
        class_map[i] = folder.split("/")[-1]
        # gather all files
        train_files = glob.glob(os.path.join(folder, "train/*"))
        test_files = glob.glob(os.path.join(folder, "test/*"))

        for f in train_files:
            train_points.append(trimesh.load(f).sample(num_points)) #Breaking the image into points 
            train_labels.append(i)                                  # and add it to array

        for f in test_files:
            test_points.append(trimesh.load(f).sample(num_points))
            test_labels.append(i)

    return (
        np.array(train_points),
        np.array(test_points),
        np.array(train_labels),
        np.array(test_labels),
        class_map,
    )

In [7]:
NUM_POINTS = num_points

train_points, test_points, train_labels, test_labels, CLASS_MAP = parse_dataset(
    NUM_POINTS
)

processing class: bathtub
processing class: monitor


In [8]:
def augment(points, label):                                                   
    # jitter points                                                           
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    # shuffle points
    points = tf.random.shuffle(points)
    return points, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)

def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

    def get_config(self):
        return {'num_features': self.num_features, 'l2reg': self.l2reg}
    
def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)
    
    x = conv_bn(inputs, 64)
    x = conv_bn(x, 128)
    x = conv_bn(x, 1024)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 512)
    x = dense_bn(x, 256)
    
        
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

inputs = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs, 3)

x = conv_bn(x, 64)
x = conv_bn(x, 64)
x = tnet(x, 64)
x = conv_bn(x, 64)
x = conv_bn(x, 128)
if d_max == 1024:
    x = conv_bn(x, 1024)
elif d_max == 512:
    x = conv_bn(x, 512)
elif d_max == 2048:
    x = conv_bn(x, 2048)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 512)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)

2023-12-10 12:33:32.497177: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-12-10 12:33:32.499694: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-12-10 12:33:32.516567: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-10 12:33:32.516584: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2023-12-10 12:33:32.516930: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 12:33:32.520871: I tensorflow/comp

##### TRAIN THE MODEL

In [11]:
if NUM_CLASSES == 2:
    outputs = layers.Dense(1, activation="sigmoid")(x)  # Single output neuron for binary classification
    loss_function = "binary_crossentropy"
    metrics = ["accuracy"]
else:
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)  # Multiple output neurons for multi-class
    loss_function = "sparse_categorical_crossentropy"
    metrics = ["sparse_categorical_accuracy"]

# Define the model
model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
model.summary()

# Compile the model
model.compile(
    loss=loss_function,
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=metrics
)

# Train the model
model.fit(train_dataset, epochs=20, validation_data=test_dataset)

weights_chair = model.get_weights()

# # Construct the file name using all parameters
# weights_filename = f"weights_{chair_t}_{NUM_CLASSES}_{num_points}_{batch}_{epochs}_{d_max}.npy"

model_path = '/home/DAVIDSON/dmkurdydyk/FRIB_Distant_Transfer/new_model_saves'  # Change this to your desired path

weights_path = model_path
model.save_weights(weights_path)

# # Load the model
# loaded_model = keras.models.load_model(model_path, custom_objects={'OrthogonalRegularizer': OrthogonalRegularizer})

Model: "pointnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 3)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 512, 64)      256         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 64)      256         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 512, 64)      0           batch_normalization[0][0]        
___________________________________________________________________________________________